In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

from scipy.stats import f, chi2

%matplotlib inline

# MOOC Econometrics
## Training Exercise 5.5

In [12]:
data = pd.read_csv('TrainExer5-5.txt', sep='\t')
data.head()

,rep id,response,male,activity,age
0,1,1,0,0,58
1,2,1,1,0,50
2,3,1,1,0,40
3,4,1,1,0,36
4,5,1,1,0,28


We want to train model `response ~ const + male + active + age + (age/10)^2`

In [17]:
data['(age/10)^2'] = (data.age / 10.0) ** 2

resp = data.ix[:, 'response']
X = sm.add_constant(data.ix[:, ['male', 'activity', 'age', '(age/10)^2']])

model = sm.Logit(resp, X).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.650662
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               response   No. Observations:                  925
Model:                          Logit   Df Residuals:                      920
Method:                           MLE   Df Model:                            4
Date:                Sun, 22 Nov 2015   Pseudo R-squ.:                 0.06112
Time:                        20:12:12   Log-Likelihood:                -601.86
converged:                       True   LL-Null:                       -641.04
                                        LLR p-value:                 3.886e-16
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -2.4884      0.890     -2.796      0.005        -4.233    -0.744
male           0.9537      0.158      6.029      0.000         0.644     1.264
activity       0.9137      0.185      4.945      0.000         0.552     1.276
age            0.0699      0.036      1.964      0.049         0.000     0.140
(age/10)^2    -0.0687      0.034     -2.015      0.044        -0.136    -0.002
==============================================================================
"""

![](https://habrastorage.org/files/0fb/575/79d/0fb57579d3c0422f8dacdef852d89f4e.png)

Nagelkerke $R^2$

In [31]:
n = len(resp)
model_base = sm.Logit(resp, np.ones_like(resp)).fit()

Optimization terminated successfully.
         Current function value: 0.693016
         Iterations 3


In [59]:
nom = 1 - np.exp((model_base.llf - model.llf) * 2 / n)
denom = 1 - np.exp(model_base.llf * 2 / n)

nom / denom

0.10830134648510828

Log likelihood test

Test the null hypothesis $H_0: \, β_1 = β_2 = 0$ versus $H_1$ : no restrictions on $β_1$ and $β_2$, using a likelihood ratio test.

(In this case $\beta_0$ is the intercept, not $\beta_1$)

In [82]:
X_R = sm.add_constant(data.ix[:, ['age', '(age/10)^2']])
model_restricted = sm.Logit(resp, X_R).fit()

Optimization terminated successfully.
         Current function value: 0.687672
         Iterations 4


In [83]:
print 'LL under H0:', model_restricted.llf
print 'LL under H1:', model.llf

LL under H0: -636.09656203
LL under H1: -601.862358095


Expected values:

- $H_0: LL = -636.097$
- $H_1: LL = -601.862$

In [84]:
LR = -2 * (model_restricted.llf - model.llf)

m = 2

print 'observed statistic:', LR
print 'critical value:', chi2.ppf(0.95, m)
print 'p-value: ', 1 - chi2.cdf(LR, m)

observed statistic: 68.4684078698
critical value: 5.99146454711
p-value:  1.33226762955e-15


So we reject $H_0$ 